# NON-INTERACTING (HARD-CORE / U=0) CASE

Many-Body Chern Number without Integration

Koji Kudo, Haruki Watanabe, Toshikaze Kariyado, and Yasuhiro Hatsugai

Phys. Rev. Lett. 122, 146601 – Published 9 April 2019

https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.146601

In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")

HSP_T (generic function with 1 method)

In [2]:
using QuantumOptics

## Initial Parameters

In [3]:
Nx=3; Ny=3; N=Nx*Ny; q=Ny; PN=1; T_size=5;

## Create Single-Particle Operator

In [4]:
function Op_Sp(Nx, Ny, alpha, Tx, Ty)
        
    H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
    
    basis = NLevelBasis(N) 
    
    H = SparseOperator(basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

Op_Sp (generic function with 1 method)

In [5]:
# # CHECK: Operator Form
# using LinearAlgebra
# Tx=Ty=0
# op = Op_Sp(Nx, Ny, 1/q, Tx, Ty)
# eigenenergies(dense(op)) == eigvals(HSP_T(Nx, Ny, 1/q, Tx, Ty, 0))
# eigenenergies(dense(op))

## Create Many-Basis for Hard-Core Interaction

In [6]:
basis = NLevelBasis(N)
states = bosonstates(basis, PN)
basis_mb = ManyBodyBasis(basis, states)

function get_mb_op(basis_mb, op)
    H_mb2 = SparseOperator(basis_mb) 
    for m in 1:N
        for n in 1:N
            H_mb2 += op.data[m,n] * transition(basis_mb, m, n)
        end
    end
    
    return H_mb2
end

get_mb_op (generic function with 1 method)

In [7]:
# CHECK: MB Structure (PN=1)
#eigenenergies(dense(H_mb)) == eigvals(HSP_T(Nx, Ny, 1/q, Tx, Ty, 0))
# eigenenergies(dense(get_mb_op(basis_mb, op)))

In [8]:
# # Ortogonality Check !
# E, V = eigen(dense(get_mb_op(basis_mb, op)).data)
# dot(V[:,1],V[:,2])
# norm(V)

## Crete Mesh of Twisted-Angle Space

\begin{equation}
\theta_\mu = \frac{2\pi}{N_\theta}n_\mu
\end{equation}

In [21]:
dx=2*pi/T_size
dy=dx
Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
Ty=collect(range(start=0, stop=2*pi-dy, step=dy));

## Energies of Many-Body Hofstadter Model

In [10]:
# using Plots
# E = eigenenergies(dense(H_mb))
# plot(1:length(E),E,seriestype=:scatter,legend=false)

# Cherns

Berry Curvature:
\begin{equation}
\mathcal{F}(\vec{\theta}) = \log \left( U_x(\vec{\theta})U_y(\vec{\theta}+\delta_x)U_x(\vec{\theta}+\delta_y)^{-1}U_y(\vec{\theta})^{-1} \right)
\end{equation}
Link variables:
\begin{equation}
U_\mu(\vec{\theta}) = \frac{
det \left( \Phi(\vec{\theta})^\dagger \Phi(\vec{\theta}+\delta_\mu) \right) 
}{
| det \left(\Phi(\vec{\theta})^\dagger \Phi(\vec{\theta}+\delta_\mu) \right) |} , \quad \delta_x = \left(\frac{2\pi}{N_{\theta}},0 \right), \delta_y = \left(0, \frac{2\pi}{N_{\theta}}\right)
\end{equation}
Chern Number:
\begin{equation}
\mathcal{C} = \frac{1}{2\pi i} \sum_{\vec{\theta}} \mathcal{F}(\vec{\theta})
\end{equation}

In [52]:
# executeme

function Chern_Nums(Tx, Ty, n1, n2)
    
    Sum=0
    
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty])
            H_mb = get_mb_op(basis_mb, op)
            w1, v1 = eigen(dense(H_mb).data)
            i = sortperm(w1, by=real);w1 = w1[i];v1 = v1[:,i]

            if isapprox(v1[1,1], 0+0im, atol=0.001) == false
                v1 = v1 ./ v1[1,1]
                v1 = v1 / norm(v1)
            end
            
            v1 = v1[:,n1:n2]
            
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty])
            H_mb = get_mb_op(basis_mb, op)
            w2, v2 = eigen(dense(H_mb).data)
            i = sortperm(w2, by=real);w2 = w2[i];v2 = v2[:,i]

            if isapprox(v2[1,1], 0+0im, atol=0.001) == false
                v2 = v2 ./ v2[1,1]
                v2 = v2 / norm(v2)
            end
            
            v2 = v2[:,n1:n2]
            
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty]+dy)
            H_mb = get_mb_op(basis_mb, op)
            w3, v3 = eigen(dense(H_mb).data)
            i = sortperm(w3, by=real);w3 = w3[i];v3 = v3[:,i]


            if isapprox(v3[1,1], 0+0im, atol=0.001) == false
                v3 = v3 ./ v3[1,1]
                v3 = v3 / norm(v3)
            end
            
            v3 = v3[:,n1:n2]
            
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty]+dy)
            H_mb = get_mb_op(basis_mb, op)
            w4, v4 = eigen(dense(H_mb).data)
            i = sortperm(w4, by=real);w4 = w4[i];v4 = v4[:,i]

            if isapprox(v4[1,1], 0+0im, atol=0.001) == false
                v4 = v4 ./ v4[1,1]
                v4 = v4 / norm(v4)
            end
            
            v4 = v4[:,n1:n2]
            
            #----------LINK VARIABLES------------
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
            
        end
    end
    
    return 1/(2*pi*1im)*Sum
end

Chern_Nums (generic function with 1 method)

In [53]:
Chern_Nums(Tx, Ty, 1, 3)

0.9999999999999998 - 1.3804490804951803e-19im

# TEST

In [28]:
using NBInclude; using QuantumOptics; using Plots; using LinearAlgebra
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")
function Chern(Nx::Int64, Ny::Int64, alpha, TSize::Int64, PN::Int64, PlotBands::Bool, n1::Int64, n2::Int64)

    # Single Particle #
    N = Nx*Ny
    function Op_Sp(Nx, Ny, alpha, Tx, Ty)    
        H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
        basis = NLevelBasis(N) 
        H = SparseOperator(basis)
        for m in 1:N
            for n in 1:N
                H = H + H_T[m,n] * transition(basis, m, n)
            end
        end
        return H
    end
    # ---- #

    # Fermionic Many-Body #
    basis = NLevelBasis(N)
    states = fermionstates(basis, PN)
    basis_mb = ManyBodyBasis(basis, states)
    function get_mb_op(basis_mb, op)
        H_mb2 = SparseOperator(basis_mb) 
        for m in 1:N
            for n in 1:N
                H_mb2 += op.data[m,n] * transition(basis_mb, m, n)
            end
        end
        return H_mb2
    end
    # ---- #
    
    # Plot #
    if PlotBands == true
        Tx=Ty=0
        op = Op_Sp(Nx, Ny, 1/q, Tx, Ty)
        Hamiltonian = get_mb_op(basis_mb, op)
        E = eigenenergies(dense(Hamiltonian))
        P = scatter(1:length(E),E,legend=false, xlabel="n", ylabel="E", title="Energy Bands")
        display(P)
    end
    # ---- #

    # Twist Angle Parameter Space #
    dx=2*pi/TSize
    dy=dx
    Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
    Ty=collect(range(start=0, stop=2*pi-dy, step=dy))
    # ---- #

    # Link Variable and Berry Curvature #
    Sum=0
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty])
            H_mb = get_mb_op(basis_mb, op)
            w1, v1 = eigen(dense(H_mb).data)
            i = sortperm(w1, by=real);w1 = w1[i];v1 = v1[:,i]
            v1 = v1[:,n1:n2]  
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty])
            H_mb = get_mb_op(basis_mb, op)
            w2, v2 = eigen(dense(H_mb).data)
            i = sortperm(w2, by=real);w2 = w2[i];v2 = v2[:,i]
            v2 = v2[:,n1:n2]
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty]+dy)
            H_mb = get_mb_op(basis_mb, op)
            w3, v3 = eigen(dense(H_mb).data)
            i = sortperm(w3, by=real);w3 = w3[i];v3 = v3[:,i]
            v3 = v3[:,n1:n2]
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty]+dy)
            H_mb = get_mb_op(basis_mb, op)
            w4, v4 = eigen(dense(H_mb).data)
            i = sortperm(w4, by=real);w4 = w4[i];v4 = v4[:,i]
            v4 = v4[:,n1:n2]
            #----------LINK VARIABLES------------
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F 
        end
    end
    # ---- #
    
    return print("Chern Number of [",n1,",",n2,"] band is: ",1/(2*pi*1im)*Sum)
end

Chern (generic function with 1 method)

$\nu = \frac{N}{N_\phi}, N_\phi = N_y$

In [29]:
PN=1

1

In [30]:
TSize = 5
PlotBands = false
n1 = 1
n2 = 9
Chern(Nx::Int64, Ny::Int64, alpha, TSize::Int64, PN::Int64, PlotBands::Bool, n1::Int64, n2::Int64)

Chern Number of [1,3] band is: 0.9999999999999993 - 2.1189893385618477e-16im

# Check with "Hofstadter FBA Chern Numbers" file

In [41]:
n1 = 1
n2 = 9

9

In [54]:
tx = ty = 1
op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty])
H_mb = get_mb_op(basis_mb, op)
w1, v1 = eigen(dense(H_mb).data)
i = sortperm(w1, by=real);w1 = w1[i];v1 = v1[:,i]

if isapprox(v1[1,1], 0+0im, atol=0.001) == false
    v1 = v1 ./ v1[1,1]
    v1 = v1 / norm(v1)
end

v1 = v1[:,n1:n2]

#------------------------------------
op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty])
H_mb = get_mb_op(basis_mb, op)
w2, v2 = eigen(dense(H_mb).data)
i = sortperm(w2, by=real);w2 = w2[i];v2 = v2[:,i]

if isapprox(v2[1,1], 0+0im, atol=0.001) == false
    v2 = v2 ./ v2[1,1]
    v2 = v2 / norm(v2)
end

v2 = v2[:,n1:n2]

#------------------------------------
op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty]+dy)
H_mb = get_mb_op(basis_mb, op)
w3, v3 = eigen(dense(H_mb).data)
i = sortperm(w3, by=real);w3 = w3[i];v3 = v3[:,i]


if isapprox(v3[1,1], 0+0im, atol=0.001) == false
    v3 = v3 ./ v3[1,1]
    v3 = v3 / norm(v3)
end

v3 = v3[:,n1:n2]

#------------------------------------
op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty]+dy)
H_mb = get_mb_op(basis_mb, op)
w4, v4 = eigen(dense(H_mb).data)
i = sortperm(w4, by=real);w4 = w4[i];v4 = v4[:,i]

if isapprox(v4[1,1], 0+0im, atol=0.001) == false
    v4 = v4 ./ v4[1,1]
    v4 = v4 / norm(v4)
end

v4 = v4[:,n1:n2]

#----------LINK VARIABLES------------
U1=det(adjoint(v1)*v2)
U1=U1/abs(U1)
U2=det(adjoint(v2)*v4)
U2=U2/abs(U2)
U3=det(adjoint(v3)*v4)
U3=U3/abs(U3)
U4=det(adjoint(v1)*v3)
U4=U4/abs(U4)

#----------BERRY CURVATURE-----------
F=log(U1*U2*1/U3*1/U4)

-2.220446049250312e-16 - 4.719912066503473e-16im

In [56]:
eigen(v1)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
9-element Vector{ComplexF64}:
  -0.32801465840075295 - 0.059308473132833606im
   -0.2725222976400102 + 0.19194454511686704im
  -0.16199553023518315 - 0.29132208858054826im
 -0.005814093810660865 + 0.33328262394591174im
   0.10052266572462915 - 0.3178148907567513im
    0.1744745914924424 + 0.28402416804676417im
   0.26830488330394775 - 0.1977968672814765im
   0.29952124544228703 + 0.14628101257447026im
   0.32959547074555917 + 0.049778878805416954im
vectors:
9×9 Matrix{ComplexF64}:
   -0.151638-0.124449im   …       0.23851+0.293154im
 -0.00748168+0.0372432im        0.0144498+0.139226im
    0.425551+0.0292314im         0.354841+0.360739im
   -0.298878+0.0786868im         0.540588+0.0im
   -0.222109-0.331671im         -0.339208-0.0479291im
  -0.0513805-0.0817481im  …     -0.202349-0.0399316im
    -0.12425+0.192471im      -0.000833344-0.0591432im
    0.503442+0.0im               0.125173-0.151831im
  -0.0529859+0

In [43]:
w4

9-element Vector{Float64}:
 -2.546639394327156
 -2.5466393943271535
 -2.5466393943271526
  0.20750036611082523
  0.20750036611082612
  0.20750036611082656
  2.3391390282163353
  2.3391390282163353
  2.3391390282163353

In [49]:
v1[:,2:2]

9×1 Matrix{ComplexF64}:
 0.052831216351296846 - 0.09150635094610983im
 0.052831216351296846 + 0.09150635094610986im
   0.3943375672974066 + 3.294430955222645e-18im
 -0.10566243270259382 + 2.3461773124865155e-17im
 -0.10566243270259379 - 2.3461773124865134e-17im
   0.3943375672974066 + 3.294430955222645e-18im
  -0.1971687836487033 - 0.34150635094610976im
 -0.19716878364870338 + 0.3415063509461098im
   0.5773502691896254 - 0.0im

In [45]:
eigen(v1)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
9-element Vector{ComplexF64}:
 -0.9237821843730506 - 0.3829183670639908im
 -0.9127317674854903 + 0.4085593232601819im
 -0.5215342306419987 - 0.8532303594391482im
 -0.2623864043148756 + 0.9649628877996882im
 0.23431505737350233 - 0.9721607140221473im
  0.5029485585228823 + 0.8643163468775512im
  0.5149311226021472 - 0.8572315550513134im
  0.8941587112056492 - 0.44775015262426376im
   0.994590666507338 + 0.10387206600664475im
vectors:
9×9 Matrix{ComplexF64}:
  -0.233362-0.249872im     0.123237-0.0983636im   …   -0.254675+0.310808im
   0.432492+0.0im          0.253115+0.221968im        0.0356247-0.128093im
  -0.162924-0.167249im    -0.460904+0.00584482im        0.37181-0.115709im
   0.321713+0.147516im    0.0935112-0.407792im          0.18966-0.156605im
   0.175821-0.28191im    -0.0480294+0.0996459im        0.624119+0.0im
   -0.35168-0.0147907im   -0.125152-0.127691im    …  -0.0316598+0.210359im
   0.233853-0.24